In [3]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00


In [4]:
import numpy as np
from sklearn.svm import SVC
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset



In [5]:
# Load BERT model and tokenizer
model = TFAutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
# Load emotion dataset
emotions = load_dataset('SetFit/emotion')

# Tokenize the dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

# Format the dataset for TensorFlow
emotions_encoded.set_format('tf', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])



Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
# Set batch size
BATCH_SIZE = 64

# Function to order inputs
def order(inp):
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]



In [9]:
# Create train dataset
train_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['train'][:])
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# Create test dataset
test_dataset = tf.data.Dataset.from_tensor_slices(emotions_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)



In [10]:
class BERTForClassificationWithSVM(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.svm = SVC(kernel='linear', probability=True)
        self.num_classes = num_classes

    def call(self, inputs):
        bert_output = self.bert(inputs)[1]
        bert_embeddings = np.array(bert_output)
        bert_embeddings = bert_embeddings.reshape(-1, bert_embeddings.shape[-1])

        if not hasattr(self, 'svm_trained'):
            labels = inputs[1].numpy()
            self.svm.fit(bert_embeddings, labels)
            self.svm_trained = True

        predictions = self.svm.predict_proba(bert_embeddings)

        return tf.convert_to_tensor(predictions, dtype=tf.float32)


In [ ]:

# Instantiate the model
svm_classifier = BERTForClassificationWithSVM(model, num_classes=6)

# Training loop
num_epochs = 3

for epoch in range(num_epochs):
    for batch in train_dataset:
        inputs, labels = batch
        with tf.GradientTape() as tape:
            predictions = svm_classifier(inputs)
            loss = tf.keras.losses.categorical_crossentropy(labels, predictions)

        gradients = tape.gradient(loss, svm_classifier.trainable_variables)
        optimizer.apply_gradients(zip(gradients, svm_classifier.trainable_variables))


In [ ]:



for batch in test_dataset:
    inputs, labels = batch
    predictions = svm_classifier(inputs)
    predicted_classes = tf.argmax(predictions, axis=1)
    true_classes = tf.argmax(labels, axis=1)

    correct_predictions += tf.reduce_sum(tf.cast(predicted_classes == true_classes, dtype=tf.int32))
    total_samples += tf.shape(predicted_classes)[0]

